In [1]:
# !pip install tensorflow

In [2]:
import torch
import pandas as pd
import numpy as np
import tensorflow as tf
from torch.utils.data import Dataset, DataLoader
import os
from pprint import pprint
import matplotlib
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
movielens_data_file_url = (
    "http://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
)
movielens_dir = "data/ml-25m/"

ratings_file = movielens_dir + "ratings.csv"
tags_file = movielens_dir + "tags.csv"
movies_file = movielens_dir + "movies.csv"

df = pd.read_csv(ratings_file)
tags = pd.read_csv(tags_file)
movies = pd.read_csv(movies_file)

encoder = LabelEncoder()
movies["movieId"] = encoder.fit_transform(movies["movieId"])
df["movieId"] = encoder.transform(df["movieId"])


In [7]:
df

,userId,movieId,rating,timestamp
0,1,292,5.0,1147880044
1,1,302,3.5,1147868817
2,1,303,5.0,1147868828
3,1,654,5.0,1147878820
4,1,878,3.5,1147868510
...,...,...,...,...
25000090,162541,11361,4.5,1240953372
25000091,162541,11928,2.5,1240951998
25000092,162541,11975,2.0,1240950697
25000093,162541,12221,4.0,1240953434


In [4]:
max_user_id = df.userId.max()+1
max_movie_id = df.movieId.max()+1

In [5]:
class movieDataset(Dataset):
    def __init__(self,df):
        self.data = df

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.data[idx][0],self.data[idx][1]
    
training_data = movieDataset(df[["userId","movieId"]].values)
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

In [6]:
# movie_model = tf.keras.Sequential([
#   tf.keras.layers.StringLookup(
#       vocabulary=unique_movie_titles, mask_token=None),
#   tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
# ])

# user_model = tf.keras.Sequential([
#   tf.keras.layers.StringLookup(
#       vocabulary=unique_user_ids, mask_token=None),
#   # We add an additional embedding to account for unknown tokens.
#   tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
# ])

# class MovielensModel(tfrs.Model):

#   def __init__(self, user_model, movie_model):
#     super().__init__()
#     self.movie_model: tf.keras.Model = movie_model
#     self.user_model: tf.keras.Model = user_model
#     self.task: tf.keras.layers.Layer = task

#   def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
#     # We pick out the user features and pass them into the user model.
#     user_embeddings = self.user_model(features["user_id"])
#     # And pick out the movie features and pass them into the movie model,
#     # getting embeddings back.
#     positive_movie_embeddings = self.movie_model(features["movie_title"])

#     # The task computes the loss and the metrics.
#     return self.task(user_embeddings, positive_movie_embeddings)

In [7]:
query = np.random.rand(8,2)
candidate = np.random.rand(8,2)

In [8]:
class rankingTask():
    def __init__(self):
        self.loss = torch.nn.CrossEntropyLoss(reduction="sum")
        pass
    
    def __call__(self, query_embeddings, candidate_embeddings):
        scores = torch.matmul(
            query_embeddings, torch.transpose(candidate_embeddings,0,1)
        )
        
        num_queries, num_candidates = scores.shape

        labels = torch.range(0, num_queries-1, dtype=int)
    
    
        loss = self.loss(input=scores, target=labels)
        
        return loss

    
class MovieModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.user_model = torch.nn.Embedding(max_user_id, 64)
        self.movie_model = torch.nn.Embedding(max_movie_id, 64)
        self.task = rankingTask()

    def forward(self, x):
        user, movie = x
        user = self.user_model(user)
        movie = self.movie_model(movie)
        
        loss = self.task(user,movie)
        return loss
        

task = rankingTask()
query_torch = torch.tensor(query)
candidate_torch = torch.tensor(candidate)
loss = task(query_torch, candidate_torch)
pprint(loss)
model = MovieModel()

tensor(17.1879, dtype=torch.float64)


<ipython-input-8-22cd094ac3b7>:13: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  labels = torch.range(0, num_queries-1, dtype=int)


In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in tqdm(enumerate(train_dataloader, 0), total=len(train_dataloader)):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        loss = model(data)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

  0%|          | 0/1576 [00:00<?, ?it/s]

<ipython-input-8-22cd094ac3b7>:13: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  labels = torch.range(0, num_queries-1, dtype=int)
/home/jiwidi/miniconda3/envs/cuda/lib/python3.8/site-packages/torch/autograd/__init__.py:154: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272068694/work/c10/cuda/CUDAFunctions.cpp:112.)
  Variable._execution_engine.run_backward(


[1,   200] loss: 108.018
[1,   400] loss: 87.670
[1,   600] loss: 74.773
[1,   800] loss: 67.085
[1,  1000] loss: 60.749
[1,  1200] loss: 56.718
[1,  1400] loss: 53.482


  0%|          | 0/1576 [00:00<?, ?it/s]

[2,   200] loss: 46.153
[2,   400] loss: 44.518
[2,   600] loss: 43.108
[2,   800] loss: 42.586
[2,  1000] loss: 40.991
[2,  1200] loss: 40.273
[2,  1400] loss: 39.147


...

[8,   200] loss: 26.557
[8,   400] loss: 26.805
[8,   600] loss: 26.953
[8,   800] loss: 26.875
[8,  1000] loss: 27.070
[8,  1200] loss: 27.018
[8,  1400] loss: 27.019


  0%|          | 0/1576 [00:00<?, ?it/s]

[9,   200] loss: 26.080
[9,   400] loss: 26.202
[9,   600] loss: 26.511
[9,   800] loss: 26.568
[9,  1000] loss: 26.499
[9,  1200] loss: 26.531
[9,  1400] loss: 26.614


  0%|          | 0/1576 [00:00<?, ?it/s]

[10,   200] loss: 25.670
[10,   400] loss: 25.853
[10,   600] loss: 25.978
[10,   800] loss: 26.177
[10,  1000] loss: 26.173
[10,  1200] loss: 26.261
[10,  1400] loss: 26.327
Finished Training


# Bruteforce

In [1]:
import torch
torch.range(1,5), torch.arange(1,5)

<ipython-input-1-36632ae7d9bc>:2: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  torch.range(1,5), torch.arange(1,5)


(tensor([1., 2., 3., 4., 5.]), tensor([1, 2, 3, 4]))

In [10]:
class BruteForceLayer(torch.nn.Module):
    def __init__(self,
               query_model,
               k: int = 14):
        super().__init__()
        
        self.query_model = query_model
        self.k = k 
    
    def index(self, candidates, identifiers=None):
        self._candidates = candidates
        
    def _compute_score(self, queries, candidates):
        scores = torch.matmul(
            queries, torch.transpose(candidates,0,1)
        )
        return scores
    
    def forward(self, queries):
        
        if self.query_model is not None:
            queries = self.query_model(queries)
        
        scores = self._compute_score(queries, self._candidates)
        
        values, indices = torch.topk(scores, k=self.k)

        return values, indices
        
        
brute_layer = BruteForceLayer(model.user_model, k=14)
# brute_layer.index(training_data.map(lambda x: model.movie_model(torch.tensor(x[1]))))
unique_movies_mapped = torch.stack(list(map(lambda x: model.movie_model(torch.tensor(x)), df.movieId.unique())))

brute_layer.index(unique_movies_mapped)
brute_layer.query_model
brute_layer(torch.tensor([42]))

(tensor([[3.2733, 2.2647, 2.2531, 2.2468, 2.2326, 2.1827, 2.1510, 2.1282, 2.0116,
          2.0090, 2.0059, 1.9886, 1.9786, 1.9784]], grad_fn=<TopkBackward0>),
 tensor([[1445, 4960, 4742, 7364, 7076, 3267, 7707, 3785, 4989, 6691,  186, 8166,
          6770, 2877]]))

In [12]:
out = brute_layer(torch.tensor([42]))[1].tolist()[0]

movies[movies['movieId'].isin(out)]

,movieId,title,genres
186,186,Boys on the Side (1995),Comedy|Drama
1445,1445,"Breakfast Club, The (1985)",Comedy|Drama
2877,2877,Easy Money (1983),Comedy
3267,3267,Cries and Whispers (Viskningar och rop) (1972),Drama
3785,3785,Changing Lanes (2002),Drama|Thriller
4742,4742,Beauty and the Beast (La belle et la bête) (1946),Drama|Fantasy
4960,4960,Fail-Safe (1964),Drama|Thriller|War
4989,4989,Look Who's Talking Too (1990),Comedy|Romance
6691,6691,Descent (2007),Drama|Thriller
6770,6770,"Incredible Hulk, The (2008)",Action|Sci-Fi


In [ ]:
# query_tf = tf.convert_to_tensor(query)
# candidate_tf = tf.convert_to_tensor(candidate)


# scores = tf.linalg.matmul(
#         query_tf, candidate_tf, transpose_b=True)

# loss = tf.keras.losses.CategoricalCrossentropy(
#         from_logits=True, reduction=tf.keras.losses.Reduction.SUM)

# num_queries = tf.shape(scores)[0]
# num_candidates = tf.shape(scores)[1]

# labels = tf.eye(num_queries, num_candidates)

# scores.shape, labels.shape

# loss = loss(y_true=labels, y_pred=scores)
# loss
# import torch
# import tensorflow as tf

# query = np.array([[2.0,2,],[4,4]])

# query_torch = torch.tensor(query)
# eye_torch = torch.range(0,1, dtype=int)
# torch_loss = torch.nn.CrossEntropyLoss(reduction="sum")
# print(f"Loss with torch {torch_loss(query_torch, eye_torch)}")



# query_tf = tf.convert_to_tensor(query)
# eye_tf = tf.eye(2,2)
# tf_loss = tf.keras.losses.CategoricalCrossentropy(
#         from_logits=True, reduction=tf.keras.losses.Reduction.SUM)


# print(f"Loss with tf {tf_loss(y_pred=query_tf, y_true=eye_tf)}")


